In [ ]:
from queue import PriorityQueue
from itertools import permutations

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def best_first_search(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    frontier = PriorityQueue()
    frontier.put(Node(start))
    visited = set()

    while not frontier.empty():
        current_node = frontier.get()
        if current_node.position == end:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        visited.add(current_node.position)

        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_node.position[0] + dx, current_node.position[1] + dy)
            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in visited:
                new_node = Node(new_pos, current_node)
                new_node.h = heuristic(new_pos, end)
                new_node.f = new_node.h
                frontier.put(new_node)
                visited.add(new_pos)

    return None

def find_shortest_path_through_goals(maze, start, goals):
    shortest_path, shortest_length = None, float("inf")

    for perm in permutations(goals):
        current_path, current_start, total_length = [], start, 0

        for goal in perm:
            path_segment = best_first_search(maze, current_start, goal)
            if not path_segment:
                break
            total_length += len(path_segment) - 1
            current_start = goal
            current_path.extend(path_segment[:-1])

        current_path.append(current_start)
        if total_length < shortest_length:
            shortest_length, shortest_path = total_length, current_path

    return shortest_path

maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start, goals = (0, 0), [(2, 3), (4, 4), (1, 1)]
optimal_path = find_shortest_path_through_goals(maze, start, goals)
print(optimal_path)

[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4)]


In [1]:
import random
import time
from queue import PriorityQueue

class AStarDynamic:
    def __init__(self, graph, heuristic):
        self.graph = graph
        self.heuristic = heuristic

    def heuristic_function(self, node):
        return self.heuristic[node]

    def a_star(self, start, goal):
        open_list = PriorityQueue()
        open_list.put((0, start))
        came_from = {start: None}
        g_costs = {start: 0}

        while not open_list.empty():
            _, current = open_list.get()
            if current == goal:
                return self.reconstruct_path(came_from, current)

            for neighbor, cost in self.graph[current].items():
                new_g_cost = g_costs[current] + cost
                if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                    g_costs[neighbor] = new_g_cost
                    f_cost = new_g_cost + self.heuristic_function(neighbor)
                    open_list.put((f_cost, neighbor))
                    came_from[neighbor] = current

        return None

    def reconstruct_path(self, came_from, current):
        path = []
        while current:
            path.append(current)
            current = came_from[current]
        return path[::-1]

    def update_edge_costs(self):
        for node in self.graph:
            for neighbor in self.graph[node]:
                self.graph[node][neighbor] = random.randint(1, 15)

graph = {
    'A': {'B': 4, 'C': 3},
    'B': {'E': 12, 'F': 5},
    'C': {'D': 7, 'E': 10},
    'D': {'E': 2},
    'E': {'G': 5},
    'F': {'G': 16},
    'G': {}
}

heuristic = {'A': 14, 'B': 12, 'C': 11, 'D': 6, 'E': 4, 'F': 11, 'G': 0}

astar_dynamic = AStarDynamic(graph, heuristic)
start, goal = 'A', 'G'

while True:
    path = astar_dynamic.a_star(start, goal)
    print(path)
    time.sleep(5)
    astar_dynamic.update_edge_costs()

['A', 'C', 'D', 'E', 'G']
['A', 'C', 'E', 'G']
['A', 'C', 'E', 'G']
['A', 'B', 'F', 'G']
['A', 'B', 'E', 'G']
['A', 'B', 'F', 'G']
['A', 'B', 'E', 'G']
['A', 'C', 'E', 'G']


KeyboardInterrupt: 

In [ ]:
class DeliveryRouteGBFS:
    def __init__(self, graph, heuristic, time_windows):
        self.graph = graph
        self.heuristic = heuristic
        self.time_windows = time_windows

    def greedy_bfs(self, start, deliveries):
        route, current_node, visited = [], start, set()
        while deliveries:
            deliveries.sort(key=lambda x: (self.heuristic[x], self.time_windows[x]))
            next_delivery = deliveries.pop(0)
            route.append(next_delivery)
            visited.add(next_delivery)
            current_node = next_delivery
        return route

graph = {
    'Warehouse': {'A': 5, 'B': 2, 'C': 8},
    'A': {'D': 6, 'E': 3},
    'B': {'F': 4, 'G': 7},
    'C': {'H': 5},
    'D': {}, 'E': {}, 'F': {}, 'G': {}, 'H': {}
}

heuristic = {'Warehouse': 10, 'A': 7, 'B': 5, 'C': 9, 'D': 3, 'E': 2, 'F': 4, 'G': 6, 'H': 8}
time_windows = {'A': 3, 'B': 2, 'C': 5, 'D': 1, 'E': 4, 'F': 2, 'G': 3, 'H': 6}
deliveries = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

gbfs_route = DeliveryRouteGBFS(graph, heuristic, time_windows)
route = gbfs_route.greedy_bfs('Warehouse', deliveries)
print(route)

['E', 'D', 'F', 'B', 'G', 'A', 'H', 'C']


In [ ]:
import random
import time
from queue import PriorityQueue

class AStarTraffic:
    def __init__(self, graph, heuristic):
        self.graph = graph
        self.heuristic = heuristic

    def heuristic_function(self, node, goal):
        return self.heuristic[node]

    def a_star(self, start, goal):
        open_list = PriorityQueue()
        open_list.put((0, start))
        came_from = {start: None}
        g_costs = {start: 0}
        while not open_list.empty():
            _, current = open_list.get()
            if current == goal:
                return self.reconstruct_path(came_from, current)
            for neighbor, cost in self.graph[current].items():
                new_g_cost = g_costs[current] + cost
                if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                    g_costs[neighbor] = new_g_cost
                    f_cost = new_g_cost + self.heuristic_function(neighbor, goal)
                    open_list.put((f_cost, neighbor))
                    came_from[neighbor] = current
        return None

    def reconstruct_path(self, came_from, current):
        path = []
        while current:
            path.append(current)
            current = came_from[current]
        return path[::-1]

    def update_traffic_conditions(self):
        for node in self.graph:
            for neighbor in self.graph[node]:
                self.graph[node][neighbor] = random.randint(1, 20)
        print("Traffic conditions updated:", self.graph)

city_graph = {
    'Home': {'A': 5, 'B': 2},
    'A': {'C': 6, 'D': 3},
    'B': {'E': 4, 'F': 7},
    'C': {'G': 5},
    'D': {'G': 8},
    'E': {'G': 10},
    'F': {'G': 3},
    'G': {}
}

heuristic = {'Home': 12, 'A': 9, 'B': 7, 'C': 6, 'D': 4, 'E': 8, 'F': 5, 'G': 0}

astar_traffic = AStarTraffic(city_graph, heuristic)
start, goal = 'Home', 'G'
while True:
    path = astar_traffic.a_star(start, goal)
    print("Fastest Route:", path)
    time.sleep(5)
    astar_traffic.update_traffic_conditions()